Lo script seguente itera in ordine alfabetico i file inseriti nella cartella "files" e, a seconda del tipo, li sposta nella relativa sottocartella. Se la sottocartella non esiste lo script la crea automaticamente. Viene anche creato un file "recap.csv" in cui ogni volta che viene spostato un file viene aggiornato con il relativo nome, tipo e dimensione.

In [4]:
import os
import shutil
import csv

In [5]:
#mi sposto nella cartella files
os.chdir('files')

In [6]:
#creo le sottocartelle
for dirname in ['audio', 'docs', 'images']:
    if not os.path.exists(dirname):#...se queste non esistono
        os.makedirs(dirname)

#creo liste formati accettati
img_exts = ['.jpeg', '.jpg', '.png']
doc_exts = ['.txt', '.odt']
audio_exts = ['.mp3']

#se il file di recap non esiste creo il file di recap, se esiste già lo apro
if not os.path.exists('recap.csv'):
    recap = open('recap.csv', 'w', newline='')#lo creo: 'w' sta per 'write'
    writer = csv.writer(recap)
    writer.writerow(['name', 'type', 'size (B)'])#scrivo header del csv
else:
    recap = open('recap.csv', 'a', newline='')#lo apro in 'append' con 'a'
    writer = csv.writer(recap)

In [7]:
#creo le sottocartelle...
for dirname in ['audio', 'docs', 'images']:
    if not os.path.exists(dirname):#...se queste non esistono
        os.makedirs(dirname)

#creo liste formati accettati
img_exts = ['.jpeg', '.jpg', '.png']
doc_exts = ['.txt', '.odt']
audio_exts = ['.mp3']

#se il file di recap...
if not os.path.exists('recap.csv'):#...non esiste
    recap = open('recap.csv', 'w', newline='')#lo creo: 'w' sta per 'write'
    writer = csv.writer(recap)
    writer.writerow(['name', 'type', 'size (B)'])#scrivo header del csv
else:#...esiste già
    recap = open('recap.csv', 'a', newline='')#lo apro in 'append' con 'a'
    writer = csv.writer(recap)
    
#itero dentro cartella corrente (cioè files: mi sono spostato dentro all'inizio)
for f in sorted(os.listdir('.')):
    
    #separo il nome del file dall'estensione
    name, ext = os.path.splitext(f)
    
    if ext in img_exts:#check estensione
        size = os.path.getsize(f)#calcolo dimensione in byte
        print(f'{name} type:image size:{size}B')#stampo info
        writer.writerow([name, 'image', size])#aggiorno recap
        shutil.move(f, 'images/')#sposto in sottocartella
        
    elif ext in doc_exts:
        size = os.path.getsize(f)
        print(f'{name} type:doc size:{size}B')
        writer.writerow([name, 'doc', size])
        shutil.move(f, 'docs/')
        
    elif ext in audio_exts:
        size = os.path.getsize(f)
        print(f'{name} type:audio size:{size}B')
        writer.writerow([name, 'audio', size])
        shutil.move(f, 'audio/')
        
    else:
        print('Formato non riconosciuto.')
    
recap.close()

Formato non riconosciuto.
Formato non riconosciuto.
Formato non riconosciuto.
Formato non riconosciuto.


Il seguente script itera invece sulla cartella immagini e costruisce una tabella riassuntiva contentente le informazioni di colore dei file.

In [8]:
import numpy as np
from PIL import Image
from tabulate import tabulate

In [9]:
table = []#creo tabella per tabulate

#ciclo dentro sottocartella images,carico l'immagine e la trasformo in numpy array
for f in sorted(os.listdir('images')):
    
    img = Image.open(os.path.join('images', f))#uso join per costruire nome file relativo a sottocartella
    img_array = np.array(img)
    
    #calcolo altezza e larghezza
    pixel_y, pixel_x = img_array.shape[:2]
    
    #caso scala di grigio
    if img_array.ndim == 2:  
        mean_gs = img_array.mean()#media dei valori dell'unico livello
        mean_r, mean_g, mean_b = 0, 0, 0#non esistono altri livelli
        mean_alpha = 0
        
    #caso RGB o RGBA: ho una dimensione aggiuntiva per i canali di colore (e quello di trasparenza)
    else:
        mean_gs = 0
        #calcolo della media per ogni canale:
        #- con la prima applicazione di mean faccio la media lungo l'asse y (righe)
        #- con la seconda, faccio la media lungo l'asse x (colonne)
        #il risultato è un array di 3 (RGB) o 4 (RGBA) numeri, corrispondenti alle
        #medie dei valori di ognuno dei 3 canali di colore e, se presente, di quello di trasparenza
        channels_mean = img_array.mean(axis=0).mean(axis=0)
        if len(channels_mean) == 3:#caso RGB, solo i tre canali di colore
            mean_alpha = 0    
            mean_r, mean_g, mean_b = channels_mean
        else:#caso RGBA, c'è anche il canale di trasparenza
            mean_r, mean_g, mean_b, mean_alpha = channels_mean
    
    #separo nome ed estensione
    name, ext = os.path.splitext(f)
    #costruisco riga della tabella tabulate
    row = [name, pixel_y, pixel_x, mean_gs, mean_r, mean_g, mean_b, mean_alpha]
    #aggiungo riga a tabella
    table += [row]

#stampo tabella tabulate
print(tabulate(table,
               headers=['name', 'height', 'width', 'grayscale', 'R', 'G', 'B', 'ALPHA'],
               tablefmt='fancy_grid',
               floatfmt='.2f'))

╒════════╤══════════╤═════════╤═════════════╤════════╤════════╤════════╤═════════╕
│ name   │   height │   width │   grayscale │      R │      G │      B │   ALPHA │
╞════════╪══════════╪═════════╪═════════════╪════════╪════════╪════════╪═════════╡
│ Casa   │     1080 │    1920 │           0 │ 109.71 │ 111.40 │ 108.07 │       0 │
╘════════╧══════════╧═════════╧═════════════╧════════╧════════╧════════╧═════════╛
